In [18]:
import requests

resp = requests.get('https://codecov.io/api/gh/rails/rails')

In [19]:
resp.json()


{'commits': [],
 'repo': {'using_integration': None,
  'name': 'rails',
  'language': 'ruby',
  'deleted': False,
  'activated': False,
  'private': False,
  'updatestamp': '2019-05-13T23:12:28.345522+00:00',
  'branch': 'master',
  'active': None,
  'service_id': '8514',
  'image_token': 'nhum7hTgQ0'},
 'meta': {'status': 200},
 'owner': {'username': 'rails',
  'name': None,
  'service': 'github',
  'updatestamp': None,
  'avatar_url': None,
  'service_id': '4223'}}

In [63]:
from github import Github

g = Github("token")
result = g.search_repositories(query="language:python stars:>5000 is:public")

In [64]:
repo_covs = []
for repo in result:
    contents = g.get_repo(repo.full_name).get_contents("")
    for content in contents:
        if content.path == "codecov.yml" or content.path == ".codecov.yml":
            repo_covs.append(repo)

In [65]:
len(repo_covs)

35

In [156]:
def get_coverage(full_name):
    resp = requests.get('https://codecov.io/api/gh/%s' % full_name)
    df = json_normalize(resp.json()['commits'])
    df = df[df['timestamp'] == df['timestamp'].max()]
    if len(df) == 0:
        return 0
    return df['totals.c'][0]

In [157]:
repos = []
for r in repo_covs[:10]:
    repo = {
        'id': r.id,
        'name': r.name,
        'full_name': r.full_name,
        'description': r.description,
        'git_url': r.git_url,
        'commits': r.get_commits().totalCount,
        'subscribers_count': r.subscribers_count,
        'stargazers_count': r.stargazers_count,
        'watchers_count': r.watchers_count,
        'open_issues_count': r.open_issues_count,
        'forks_count': r.forks_count,
        'master_branch': r.master_branch,
        'language': r.language,
        'size': r.size,
        'test_coverage': get_coverage(r.full_name)
    }
    repos.append(repo)

In [158]:
df = pd.DataFrame(repos)
df

,commits,description,forks_count,full_name,git_url,id,language,master_branch,name,open_issues_count,size,stargazers_count,subscribers_count,test_coverage,watchers_count
0,23996,scikit-learn: machine learning in Python,17124,scikit-learn/scikit-learn,git://github.com/scikit-learn/scikit-learn.git,843222,Python,None,scikit-learn,1934,102882,35049,2245,96.27749,35049
1,7074,"Scrapy, a fast high-level web crawling & scrap...",7725,scrapy/scrapy,git://github.com/scrapy/scrapy.git,529502,Python,None,scrapy,796,17005,32781,1838,85.42813,32781
2,9198,Certbot is EFF's tool to obtain certs from Let...,2557,certbot/certbot,git://github.com/certbot/certbot.git,26516210,Python,None,certbot,763,22010,25011,884,96.85139,25011
3,19210,:house_with_garden: Open source home automatio...,6858,home-assistant/home-assistant,git://github.com/home-assistant/home-assistant...,12888993,Python,None,home-assistant,1169,108794,23533,1038,94.07169,23533
4,25384,Sentry is cross-platform application monitorin...,2365,getsentry/sentry,git://github.com/getsentry/sentry.git,873328,Python,None,sentry,862,108124,20791,651,86.18086,20791
5,19335,Flexible and powerful data analysis / manipula...,7715,pandas-dev/pandas,git://github.com/pandas-dev/pandas.git,858127,Python,None,pandas,2962,156270,19497,1010,91.67949,19497
6,2523,A code-completion engine for Vim,2131,Valloric/YouCompleteMe,git://github.com/Valloric/YouCompleteMe.git,4037197,Python,None,YouCompleteMe,48,34846,19025,598,94.56838,19025
7,4067,Tornado is a Python web framework and asynchro...,4932,tornadoweb/tornado,git://github.com/tornadoweb/tornado.git,301742,Python,None,tornado,157,8424,17725,1075,87.70290,17725
8,8011,An interactive TLS-capable intercepting HTTP p...,1947,mitmproxy/mitmproxy,git://github.com/mitmproxy/mitmproxy.git,519832,Python,None,mitmproxy,238,45066,14983,531,88.99817,14983
9,8001,Web APIs for Django. ⚡️,4178,encode/django-rest-framework,git://github.com/encode/django-rest-framework.git,1431547,Python,None,django-rest-framework,214,41385,14028,522,95.75400,14028
